In [ ]:
# Third-party libraries
import matplotlib.pyplot as plt
import mujoco
import numpy as np
from mujoco import viewer

# import prebuilt robot phenotypes
from ariel.body_phenotypes.robogen_lite.prebuilt_robots.gecko import gecko
from ariel.simulation.environments.olympic_arena import OlympicArena
# Local libraries
from ariel.simulation.tasks.gait_learning import xy_displacement
from ariel.utils.renderers import video_renderer
from ariel.utils.video_recorder import VideoRecorder


def genome_move(model, data, to_track, genome, timestep, history) -> None:
    """
    Applies joint commands from a pre-generated genome.
    
    The genome is a list of lists, where each inner list is a set of
    commands for one timestep.
    """
    delta = 0.05
    data.ctrl += genome[int(data.time / 0.002)] * delta 
    data.ctrl = np.clip(data.ctrl, -np.pi/2, np.pi/2)
# 

    # data.ctrl = genome[int(data.time / 0.002)]

    history.append(to_track[0].xpos.copy())

def create_population(model, num_steps, population):
    return [[np.random.uniform(low=-np.pi/2, high=np.pi/2, size=model.nu) for _ in range(num_steps)] for _ in range(population)]



from multiprocessing import Pool, cpu_count

# --- helper function for pool (must be top-level) ---
def _evaluate_star(args):
    return evaluate_one(*args)


def fitness_function(target_position, history):
    xt, yt, zt = target_position
    distances = [
        np.sqrt((xt - x)**2 + (yt - y)**2 + (zt - z)**2)
        for x, y, z in history
    ]
    return np.mean(distances)  # average distance instead of only final


import time as t
def evaluate_one(genome, robot_core_func, world_func, time, spawn_pos):
    mujoco.set_mjcb_control(None)
    world = world_func()
    world.spawn(robot_core_func().spec, spawn_pos)
    model = world.spec.compile()
    data = mujoco.MjData(model)

    geoms = world.spec.worldbody.find_all(mujoco.mjtObj.mjOBJ_GEOM)
    to_track = [data.bind(geom) for geom in geoms if "core" in geom.name]
    
    timestep = [0]
    history  = []
    mujoco.set_mjcb_control(lambda m,d: genome_move(m, d, to_track, genome, timestep, history))

    while data.time < time and timestep[0] < len(genome):
        mujoco.mj_step(model, data)
        timestep[0] += 1
        
    pos_data = np.array(history)
    distance_to_goal = fitness_function((5, 0, 0.5), history)
        
    return distance_to_goal


def evaluate(population, robot_core_func, world_func, time, spawn_pos, n_workers=None):
    if n_workers is None:
        n_workers = min(len(population), cpu_count() - 1)

    results_fitness = []
    total = len(population)

    with Pool(n_workers) as pool:
        args = [
            (genome, robot_core_func, world_func, time, spawn_pos)
            for genome in population
        ]

        for i, result in enumerate(pool.imap_unordered(_evaluate_star, args), 1):
            results_fitness.append(result)
            print(f"Processed genome {i}/{total} with fitness: {result}")

    return results_fitness



def parent_selection(x, f):
    x_parents, f_parents = [],[]
    
    total_f              = np.sum(f)
    
    s                    = 0.000000000001
    max_probs            = np.array([i / (total_f + s) for i in f])    
    min_probs            = 1 / (max_probs + s)
    min_probs_normalized = min_probs / (min_probs.sum() + s)
    
    for _ in range(int(len(x)/2)):
        parent_indices = np.random.choice(len(x), size=2, replace=True, p=min_probs_normalized)
        x_parents.append([x[i] for i in parent_indices])
        f_parents.append([f[i] for i in parent_indices])

    return x_parents, f_parents

def crossover(x_parents, p_crossover, alpha=0.5):
    offspring = []
    for p1, p2 in x_parents:
        if np.random.rand() > p_crossover:
            offspring.extend([p1, p2])
            continue
        
        child1, child2 = [], []
        for t in range(len(p1)):
            mix = np.random.rand(*p1[t].shape)
            c1 = alpha * p1[t] + (1 - alpha) * p2[t] + 0.1 * np.random.randn(*p1[t].shape)
            c2 = alpha * p2[t] + (1 - alpha) * p1[t] + 0.1 * np.random.randn(*p2[t].shape)
            child1.append(np.clip(c1, -np.pi/2, np.pi/2))
            child2.append(np.clip(c2, -np.pi/2, np.pi/2))
        offspring.extend([np.array(child1), np.array(child2)])
    return np.array(offspring)



def mutate(pop, mutation_rate=0.3, mutation_scale=0.5):
    for i in range(len(pop)):
        for t in range(len(pop[i])):
            if np.random.rand() < mutation_rate:
                noise = np.random.normal(0, mutation_scale, size=pop[i][t].shape)
                pop[i][t] = np.clip(pop[i][t] + noise, -np.pi/2, np.pi/2)
    return pop


def survivior_selection(x, f, x_offspring, f_offspring):
    """Select the survivors, for the population of the next generation. Returns a list of survivors and their fitness values."""

    # 1) This one looks slightly weird since in the provided code the x and f is passed as a parent pair so it needs to be flattend
    _x = []
    for i in x: _x.extend(i)
    
    _f = []
    for i in f: _f.extend(i)
    
    x_offspring = x_offspring.tolist() # For simplicity

    # 2) Get combine population
    combined_x = np.array(_x + x_offspring)
    combined_f = np.array(_f + f_offspring)
    
    # 3) Sort based on best performing
    sorted_indices = np.argsort(combined_f)
    
    # 4) Get n best performing
    n = len(_x)
    x = combined_x[sorted_indices[:n]]
    f = combined_f[sorted_indices[:n]]

    return x, f


def main(world_func, robot_func, spawn_pos = [-0.8, 0, 0.1],time = 30, population = 10, generations = 50, p_crossover = 0.5, m_rate = 0.1):
    """Main function to run the simulation with random movements."""

    mujoco.set_mjcb_control(None) # DO NOT REMOVE
    world       = world_func()
    gecko_core  = robot_func()     # DO NOT CHANGE
    world.spawn(gecko_core.spec, spawn_pos)
    model       = world.spec.compile()
    num_steps   = int(time / model.opt.timestep)

    population  = create_population(model, num_steps, population)
    population_fitness = evaluate(population, robot_func, world_func, time, spawn_pos)

    idx = np.argmin(population_fitness) 
    x0_best = population[idx]
    f0_best = population_fitness[idx]
    
    x_best = [x0_best]
    f_best = [f0_best]

    for _ in range(generations):
        parents, parents_f             = parent_selection(population, population_fitness)
        offsprings                     = crossover(parents, p_crossover)
        
        offsprings                     = mutate(offsprings, m_rate)
        f_offspring                    = evaluate(offsprings, robot_func, world_func, time, spawn_pos)

        elite_idx = np.argmin(population_fitness)
        elite = population[elite_idx]
        elite_fitness = population_fitness[elite_idx]

        population, population_fitness = survivior_selection(
            parents, parents_f, offsprings, f_offspring
        )

        # Ensure elite survives
        worst_idx = np.argmax(population_fitness)
        population[worst_idx] = elite
        population_fitness[worst_idx] = elite_fitness

        idx = np.argmin(population_fitness) # !!! SWITCHED FROM .argmax() to .argmin() FOR MINIMIZATION !!!
        xi_best = population[idx]
        fi_best = population_fitness[idx]
        print(f"Best one is {fi_best}")
        if fi_best < f_best[-1]: # !!! SWITCHED FROM > to < FOR MINIMIZATION !!!
            x_best.append(xi_best)
            f_best.append(fi_best)
        else:
            x_best.append(x_best[-1])
            f_best.append(f_best[-1])
    
    return x_best, f_best


x, a = None, None
if __name__ == "__main__":
    x, a = main(OlympicArena,gecko,time=30, population= 12, generations=80)

    plt.figure(figsize=(10, 6))
    plt.plot(a, 'm-', label='Distance to Target')

    plt.ylim(bottom = 0)
    plt.legend()
    plt.grid(True)





Processed genome 1/12 with fitness: 5.882441503246232
Processed genome 2/12 with fitness: 5.778630031215294
Processed genome 3/12 with fitness: 5.750911535073857
Processed genome 4/12 with fitness: 5.787604157309721
Processed genome 5/12 with fitness: 5.785892245970617
Processed genome 6/12 with fitness: 5.861583826584802
Processed genome 7/12 with fitness: 5.858518875308748
Processed genome 8/12 with fitness: 5.819570012144411
Processed genome 9/12 with fitness: 5.815220633123002
Processed genome 10/12 with fitness: 5.761469241876614
Processed genome 11/12 with fitness: 5.748911468216364
Processed genome 12/12 with fitness: 5.742224192419
Processed genome 1/12 with fitness: 5.865756420397602
Processed genome 2/12 with fitness: 5.860430942330086
Processed genome 3/12 with fitness: 5.756355215411878
Processed genome 4/12 with fitness: 5.814704253006732
Processed genome 5/12 with fitness: 5.802915654319567
Processed genome 6/12 with fitness: 5.734309251969095
Processed genome 7/12 with f

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/s4ddo/.local/share/uv/python/cpython-3.12.9-linux-x86_64-gnu/lib/python3.12/multiprocessin │
│ g/pool.py:856 in next                                                                            │
│                                                                                                  │
│   853 │   def next(self, timeout=None):                                                          │
│   854 │   │   with self._cond:                                                                   │
│   855 │   │   │   try:                                                                           │
│ ❱ 856 │   │   │   │   item = self._items.popleft()                                               │
│   857 │   │   │   except IndexError:                                                             │
│   858 │   │   │   │   if self._index == self._length:                                            │
│   859 │   │   │   │   │   self._pool = None                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:231                                                                                  │
│                                                                                                  │
│   228                                                                                            │
│   229 x, a = None, None                                                                          │
│   230 if __name__ == "__main__":                                                                 │
│ ❱ 231 │   x, a = main(OlympicArena,gecko,time=30, population= 12, generations=80)                │
│   232 │                                                                                          │
│   233 │   plt.figure(figsize=(10, 6))                                                            │
│   234 │   plt.plot(a, 'm-', label='Distance to Target')                                          │
│                                                                                                  │
│ in main:200                                                                                      │
│                                                                                                  │
│   197 │   │   offsprings                     = crossover(parents, p_crossover)                   │
│   198 │   │                                                                                      │
│   199 │   │   offsprings                     = mutate(offsprings, m_rate)                        │
│ ❱ 200 │   │   f_offspring                    = evaluate(offsprings, robot_func, world_func, ti   │
│   201 │   │                                                                                      │
│   202 │   │   elite_idx = np.argmin(population_fitness)                                          │
│   203 │   │   elite = population[elite_idx]                                                      │
│                                                                                                  │
│ in evaluate:92                                                                                   │
│                                                                                                  │
│    89 │   │   │   for genome in population                                                       │
│    90 │   │   ]                                                                                  │
│    91 │   │                                                                                      │
│ ❱  92 │   │   for i, result in enumerate(pool.imap_uno

In [ ]:
mujoco.set_mjcb_control(None) # DO NOT REMOVE
world = OlympicArena()
gecko_core = gecko()     # DO NOT CHANGE
world.spawn(gecko_core.spec, [-0.8, 0, 0.1])
model = world.spec.compile()
data = mujoco.MjData(model) # type: ignore
geoms = world.spec.worldbody.find_all(mujoco.mjtObj.mjOBJ_GEOM)
to_track = [data.bind(geom) for geom in geoms if "core" in geom.name]

timestep = [0]
history = []
mujoco.set_mjcb_control(lambda m,d: genome_move(m, d, to_track, x[-1], timestep, history))


viewer.launch(
    model=model,  # type: ignore
    data=data,
)



Exception in thread Thread-772 (_physics_loop):
Traceback (most recent call last):
  File "/home/s4ddo/.local/share/uv/python/cpython-3.12.9-linux-x86_64-gnu/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/home/s4ddo/Documents/Masters_Year_1/P1/EvoAI/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/home/s4ddo/.local/share/uv/python/cpython-3.12.9-linux-x86_64-gnu/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/home/s4ddo/Documents/Masters_Year_1/P1/EvoAI/.venv/lib/python3.12/site-packages/mujoco/viewer.py", line 430, in _physics_loop
    mujoco.mj_step(m, d)
  File "/tmp/ipykernel_23452/3928741407.py", line 12, in <lambda>
  File "/tmp/ipykernel_23452/3277646703.py", line 28, in genome_move
IndexError: index 15000 is out of bounds for axis 0 with size 15000
